We are going to use Deep learning for classify Protein sequnces that they are HBPs or NON-HBPS. 
Algorithm Used

* Convolutional Neural Network 1d with Embiding Layer
* Convolutional Neural Network 2d
* Siamese Neural Network
* Some Machine Learning Algorithms without Feature Engineering

# Protein Sequence Reading

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
print(tf.__version__)

1.13.1


In [3]:
import numpy as np 
import pandas as pd 
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings  
warnings.filterwarnings(action = 'ignore')  
import gensim 
from gensim.models import Word2Vec 

In [4]:
train_data1 = pd.read_csv('../input/dataset/solubility_train.csv')
dev_data1 = pd.read_csv('../input/dataset/solubility_dev.csv')
test_data = pd.read_csv('../input/dataset/solubility_test.csv')
train_data = pd.concat([train_data1,dev_data1])
train_data.head()

,Meta,Sequence,Label
0,Psol_train_0,GMILKTNLFGHTYQFKSITDVLAKANEEKSGDRLAGVAAESAEERV...,1
1,Psol_train_1,MAHHHHHHMSFFRMKRRLNFVVKRGIEELWENSFLDNNVDMKKIEY...,0
2,Psol_train_2,MGSDKIHHHHHHMEKSIQDTIHGVIKLEDWMVEIVDTPQFQRLRRI...,0
3,Psol_train_3,MEKYIHSVEDYHRLISYLENNLNYEDSVVNHVIYVIAKTGMRYGEI...,0
4,Psol_train_4,MSLTDSFTVRSIEGVCFRYPLATPVVTSFGKMLNRPAVFVRVVDED...,0


In [5]:
from sklearn.preprocessing import LabelBinarizer
import keras
# Transform labels to one-hot
lb = LabelBinarizer()
y_train = lb.fit_transform(train_data.Label)
y_train=keras.utils.to_categorical(y_train,num_classes=2,dtype='float32')
y_test = lb.fit_transform(test_data.Label)
y_test=keras.utils.to_categorical(y_test,num_classes=2,dtype='float32')

# Tokenizer
Using the ** keras** library for text processing, 
1. ** Tokenizer**: translates every character of the sequence into a number
2. **pad_sequences:** ensures that every sequence has the same length (max_length). I decided to use a maximum length of 100, which should be sufficient for most sequences. 
3. **train_test_split:** from sklearn splits the data into training and testing samples.

In [6]:
arr=[]
for i in train_data.Sequence:
    arr.append(len(i))
    
arr=np.asarray(arr)
print("Average length of string is = ",(arr.mean()))
print("Median length of string is = ",np.median(arr))

Average length of string is =  298.93048112935753
Median length of string is =  275.0


In [7]:
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

# maximum length of sequence, everything afterwards is discarded!
max_length = 280

#create and fit tokenizer
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(train_data.Sequence)
#represent input data as word rank number sequences
x_train = tokenizer.texts_to_sequences(train_data.Sequence)
x_train = sequence.pad_sequences(x_train, maxlen=max_length)
x_test = tokenizer.texts_to_sequences(test_data.Sequence)
x_test = sequence.pad_sequences(x_test, maxlen=max_length)

# Conv1D Training 

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten,Dropout,Conv2D
from keras.layers import LSTM
from keras.layers.embeddings import Embedding

embedding_dim = 64

# create the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, embedding_dim, input_length=max_length))
model.add(Conv1D(filters=256, kernel_size=2, padding='same', activation='relu'))
model.add(Dropout(0.5))

model.add(Conv1D(filters=256, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=16, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))

model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=200, batch_size=512)
#embedding_dim = 64，epochs=200, batch_size=512

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 280, 64)           1344      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 280, 256)          33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 280, 256)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 280, 256)          131328    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 280, 256)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 280, 128)          65664     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 280, 128)          0         
__________

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten,Dropout,Conv2D
from keras.layers import LSTM
from keras.layers.embeddings import Embedding

embedding_dim = 64

# create the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, embedding_dim, input_length=max_length))
model.add(Conv1D(filters=256, kernel_size=2, padding='same', activation='relu'))
model.add(Dropout(0.5))

model.add(Conv1D(filters=256, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=16, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))

model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=200, batch_size=256)
#embedding_dim = 64，epochs=200, batch_size=256

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 280, 64)           1344      
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 280, 256)          33024     
_________________________________________________________________
dropout_5 (Dropout)          (None, 280, 256)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 280, 256)          131328    
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 280, 256)          0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 280, 128)          65664     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 280, 128)          0         
__________

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten,Dropout,Conv2D
from keras.layers import LSTM
from keras.layers.embeddings import Embedding

embedding_dim = 64

# create the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, embedding_dim, input_length=max_length))
model.add(Conv1D(filters=256, kernel_size=2, padding='same', activation='relu'))
model.add(Dropout(0.5))

model.add(Conv1D(filters=256, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=16, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))

model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=200, batch_size=128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 280, 64)           1344      
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 280, 256)          33024     
_________________________________________________________________
dropout_7 (Dropout)          (None, 280, 256)          0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 280, 256)          131328    
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 280, 256)          0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 280, 128)          65664     
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 280, 128)          0         
__________

In [13]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten,Dropout,Conv2D
from keras.layers import LSTM
from keras.layers.embeddings import Embedding

embedding_dim = 64

# create the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, embedding_dim, input_length=max_length))
model.add(Conv1D(filters=512, kernel_size=2, padding='same', activation='relu'))
model.add(Dropout(0.5))

model.add(Conv1D(filters=512, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=256, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=16, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))

model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=200, batch_size=512)
#embedding_dim = 64，epochs=200, batch_size=512

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 280, 64)           1344      
_________________________________________________________________
conv1d_28 (Conv1D)           (None, 280, 512)          66048     
_________________________________________________________________
dropout_11 (Dropout)         (None, 280, 512)          0         
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 280, 512)          524800    
_________________________________________________________________
max_pooling1d_23 (MaxPooling (None, 280, 512)          0         
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 280, 256)          262400    
_________________________________________________________________
max_pooling1d_24 (MaxPooling (None, 280, 256)          0         
__________

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten,Dropout,Conv2D
from keras.layers import LSTM
from keras.layers.embeddings import Embedding

embedding_dim = 64

# create the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, embedding_dim, input_length=max_length))
model.add(Conv1D(filters=512, kernel_size=2, padding='same', activation='relu'))
model.add(Dropout(0.5))

model.add(Conv1D(filters=512, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=256, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=16, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))

model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=200, batch_size=256)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten,Dropout,Conv2D
from keras.layers import LSTM
from keras.layers.embeddings import Embedding

embedding_dim = 64

# create the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, embedding_dim, input_length=max_length))
model.add(Conv1D(filters=512, kernel_size=2, padding='same', activation='relu'))
model.add(Dropout(0.5))

model.add(Conv1D(filters=512, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=256, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=16, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))

model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=200, batch_size=128)

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
from sklearn.metrics import classification_report
import numpy as np


print(classification_report(Y_test, y_pred))

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras.callbacks import LambdaCallback
from keras.layers import Conv1D, Flatten
from keras.layers import Dense ,Dropout,BatchNormalization
from keras.models import Sequential 
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical 
from keras import regularizers
from sklearn import preprocessing
from sklearn.ensemble import  VotingClassifier
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import metrics
from sklearn import ensemble
from sklearn import gaussian_process
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn import svm
from sklearn import tree
from sklearn import discriminant_analysis
from sklearn import model_selection
from xgboost.sklearn import XGBClassifier 
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory



# Trying different Classifiers

In [ ]:
#Machine Learning Algorithm (MLA) Selection and Initialization

MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    
    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),
    
    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    XGBClassifier()    
    ]




#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Test Accuracy' ]
MLA_compare = pd.DataFrame(columns = MLA_columns)



#index through MLA and save performance to table
row_index = 0
for alg in MLA:

    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
   # cv_results = model_selection.cross_validate(alg, X_train, y_train)
    alg.fit(x_train, y_train)
    y_pred=alg.predict(x_test)
    score=metrics.accuracy_score(y_test, y_pred)
    
    MLA_compare.loc[row_index, 'MLA Test Accuracy'] =score

    
    
    row_index+=1

    

MLA_compare

MLA_compare.to_csv("classifier.csv")
#MLA_predict

# One Hot Encoder

In [ ]:
X = train_data.Sequence.astype(str).str[0:240]
X = X.values
print(len(X[1]))
print("Every String has length equal to =",len(X[1]))
# Every Sequence Length is now 280

In [ ]:
def onehot(ltr):
     return [1 if i==ord(ltr) else 0]

def onehotvec(s):
     return [onehot(c) for c in list(s.lower())]

sequence_encode=[]

for i in range(0,len(X)):
    
    X[i]=X[i].lower()
    a=onehotvec(X[i])
    a=np.asarray(a)
    sequence_encode.append(a)
    
sequence_encode=np.asarray(sequence_encode)  
print("Shape of One Hot Encoded Sequence",sequence_encode.shape)

In [ ]:
X=sequence_encode.reshape(69420,240,24469,1)
X.shape

# Conv 2D Training

In [ ]:
from keras.layers import Conv2D,LeakyReLU,MaxPooling2D
from keras.layers.core import Activation

# create the model
model = Sequential()
model.add(Conv2D(16,kernel_size = (2,2),input_shape=(96,26,1)))
model.add(Activation("relu"))
model.add(Conv2D(32,kernel_size = (2,2),input_shape=(96,26,1)))
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(2, 2),strides=2, padding='same', data_format=None))
model.add(Dropout(0.2))

model.add(Conv2D(64,kernel_size = (2,2),input_shape=(96,26,1)))
model.add(Activation("relu"))
model.add(Conv2D(82,kernel_size = (2,2),input_shape=(96,26,1)))
model.add(Activation("relu"))



model.add(Flatten())
model.add(Dense(64, activation='relu'))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30, batch_size=1)

# Siamese Neural Network (Clustring ALgorithm)

In [ ]:
# Import Keras and other Deep Learning dependencies
from keras.models import Sequential
import time
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
import seaborn as sns
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from sklearn.preprocessing import LabelBinarizer
from keras.optimizers import *
from keras.engine.topology import Layer
from keras import backend as K
from keras.regularizers import l2
K.set_image_data_format('channels_last')
import cv2
import os
from skimage import io
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf

import numpy.random as rng
from sklearn.utils import shuffle

%matplotlib inline
%load_ext autoreload
%reload_ext autoreload
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
X_train=X_train.reshape(X_train.shape[0],96,26,1)
X_test=X_test.reshape(X_test.shape[0],96,26,1)
train_groups = [X_train[np.where(y_train==i)[0]] for i in np.unique(y_train)]
test_groups = [X_test[np.where(y_test==i)[0]] for i in np.unique(y_test)]
print('train groups:', [X.shape[0] for X in train_groups])
print('test groups:', [X.shape[0] for X in test_groups])

In [ ]:
def gen_random_batch(in_groups, batch_halfsize = 8):
    out_img_a, out_img_b, out_score = [], [], []
    all_groups = list(range(len(in_groups)))
    for match_group in [True, False]:
        group_idx = np.random.choice(all_groups, size = batch_halfsize)
        out_img_a += [in_groups[c_idx][np.random.choice(range(in_groups[c_idx].shape[0]))] for c_idx in group_idx]
        if match_group:
            b_group_idx = group_idx
            out_score += [1]*batch_halfsize
        else:
            # anything but the same group
            non_group_idx = [np.random.choice([i for i in all_groups if i!=c_idx]) for c_idx in group_idx] 
            b_group_idx = non_group_idx
            out_score += [0]*batch_halfsize
            
        out_img_b += [in_groups[c_idx][np.random.choice(range(in_groups[c_idx].shape[0]))] for c_idx in b_group_idx]
            
    return np.stack(out_img_a,0), np.stack(out_img_b,0), np.stack(out_score,0)

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Activation, Flatten, Dense, Dropout
img_in = Input(shape = X_train.shape[1:], name = 'FeatureNet_ImageInput')
n_layer = img_in
for i in range(2):
    n_layer = Conv2D(8*2**i, kernel_size = (3,3), activation = 'linear')(n_layer)
    n_layer = BatchNormalization()(n_layer)
    n_layer = Activation('relu')(n_layer)
    n_layer = Conv2D(16*2**i, kernel_size = (3,3), activation = 'linear')(n_layer)
    n_layer = BatchNormalization()(n_layer)
    n_layer = Activation('relu')(n_layer)
    n_layer = MaxPool2D((2,2))(n_layer)
n_layer = Flatten()(n_layer)
n_layer = Dense(32, activation = 'linear')(n_layer)
n_layer = Dropout(0.5)(n_layer)
n_layer = BatchNormalization()(n_layer)
n_layer = Activation('relu')(n_layer)
feature_model = Model(inputs = [img_in], outputs = [n_layer], name = 'FeatureGenerationModel')
feature_model.summary()

In [ ]:
from keras.layers import concatenate
img_a_in = Input(shape = X_train.shape[1:], name = 'ImageA_Input')
img_b_in = Input(shape = X_train.shape[1:], name = 'ImageB_Input')
img_a_feat = feature_model(img_a_in)
img_b_feat = feature_model(img_b_in)
combined_features = concatenate([img_a_feat, img_b_feat], name = 'merge_features')
combined_features = Dense(16, activation = 'linear')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(4, activation = 'linear')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(1, activation = 'sigmoid')(combined_features)
similarity_model = Model(inputs = [img_a_in, img_b_in], outputs = [combined_features], name = 'Similarity_Model')
similarity_model.summary()

In [ ]:
similarity_model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['mae'])

In [ ]:
def siam_gen(in_groups, batch_size = 4):
    while True:
        pv_a, pv_b, pv_sim = gen_random_batch(train_groups, batch_size//2)
        yield [pv_a, pv_b], pv_sim
# we want a constant validation group to have a frame of reference for model performance
valid_a, valid_b, valid_sim = gen_random_batch(test_groups, 10)
loss_history = similarity_model.fit_generator(siam_gen(train_groups), 
                               steps_per_epoch = 100,
                               validation_data=([valid_a, valid_b], valid_sim),
                                              epochs = 100,
                                             verbose = True)

# Bag Of words as feature Extractor

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Split Data
lb = LabelBinarizer()
Y = lb.fit_transform(df.label)

X_train, X_test,y_train,y_test = train_test_split(df.sequences, Y, test_size = 0.2, random_state = 1)






y_test_cat=keras.utils.to_categorical(y_test)
y_train_cat=keras.utils.to_categorical(y_train)
# Create a Count Vectorizer to gather the unique elements in sequence
vect = CountVectorizer(analyzer = 'char_wb', ngram_range = (4,4))

# Fit and Transform CountVectorizer
vect.fit(X_train)
X_train_df = vect.transform(X_train)
X_test_df = vect.transform(X_test)

#Print a few of the features
print(vect.get_feature_names()[-20:])

In [ ]:
X_train_df.shape

In [ ]:
#Machine Learning Algorithm (MLA) Selection and Initialization


MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    
    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),
    
    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    XGBClassifier()    
    ]




#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Test Accuracy' ]
MLA_compare = pd.DataFrame(columns = MLA_columns)



#index through MLA and save performance to table
row_index = 0
for alg in MLA:

    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
   # cv_results = model_selection.cross_validate(alg, X_train, y_train)
    alg.fit(X_train_df.toarray(), y_train)
    y_pred=alg.predict(X_test_df.toarray())
    score=metrics.accuracy_score(y_test, y_pred)
    
    MLA_compare.loc[row_index, 'MLA Test Accuracy'] =score

    
    
    row_index+=1

    

MLA_compare
#MLA_predict

# Conclusion 

From this we can conclude that because of less number of samples we are not doing that much good by using deep learning.
I will upload my next kernal in which I will do Feature engineering and try some machine learning algorithm , Optimization Methods and ensemble technique,